# GAMMA INDEX 1D 

## Instruction 

### Use gamma index 1D to compare PDD or dose profil

In [469]:
import sys , os 
import numpy as np
import math as m
import pandas as pd

import plotly.offline as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)


#### Import values with pandas '.csv' file

In [534]:
CourbeRef="fs9e25.csv"
CourbeGATE="gs9e25.csv"
CourbeEMC='emcs9e25.csv'
df1=pd.read_csv(CourbeRef, sep=';')
df2=pd.read_csv(CourbeGATE, sep=';')
df3=pd.read_csv(CourbeEMC, sep=';')


#### Criteria of gamma Index 1D

In [523]:
DTA=3 #mm
deltaDmax=0.03 #%

#### Set True if interpolation is needed ( + set the step)
#### Set True if a Dose threshold is needed for gamma index comparison ( + set the value)

In [524]:
LinearInterpolation=True
#Point every (in mm)
#NewStep=0.1

Threshold=True
#Of Dmax
ThresholdValue=0.1

In [531]:
#Create numpy ndarray from pandas dataFrames
#You can change headers
refX=df1['pas']
refY=df1['value']
gateX=df2['pas']
gateY=df2['value']

refX=refX-(refX.max()/2)
gateX=gateX-(gateX.max()/2)


# Compute Linear interpolation for simulated values and measured values according to your parameters (change 1000 values to change number of points)
if LinearInterpolation==True:
    #A=((refX.max()-refX.min())/NewStep)
    #B=((gateX.max()-gateX.min())/NewStep)
    refX2=np.linspace(refX.min(), refX.max(), 2000)
    refY2=np.interp(refX2, refX, refY)
    gateX2=np.linspace(gateX.min(), gateX.max(), 2000)
    gateY2=np.interp(gateX2, gateX, gateY)
else:
    refX2= refX
    refY2= refY
    gateX2= gateX
    gateY2= gateY2
    

# Scaling for relative comparison
gateX2=gateX2+Xscale
gateY2=gateY2*Yscale



#create numpy ndarray for gamma comprison
deltaD=np.zeros(len(refY2))
gammaIndex=np.zeros(len(gateX2))
gammaIndexTEMP=np.zeros(len(refX2))
drTEMP=np.zeros(len(refX2))
dr=np.zeros(len(refX2))

#compute Gamma Index 1D
for i in range(len(gateX2)):
    for j in range(len(refX2)):
        dr[j]=(gateX2[i]-refX2[j])
        deltaD[j]=gateY2[i]-refY2[j]
        gammaIndexTEMP[j]=m.sqrt(((deltaD[j]**2)/deltaDmax**2) + ((dr[j]**2)/DTA**2))           
    gammaIndex[i]=gammaIndexTEMP.min()

#Compute Gamma Index Pass Rate (with threshold or not)
nok=0.0
a=0
if Threshold==True: 
    for i in range(len(gammaIndex)):

        if gateY2[i]>0.10:
            a=a+1
            if gammaIndex[i]<1:
                nok=nok+1
    print ('nombre de points testés : ' +str(a))
    print ('nombre de points passant le test (<1) : ' +str(nok))
    passRate=(nok/a)*100
else:
    nok=(gammaIndex < 1).sum()
    passRate=nok*100.0/len(gammaIndex)

#print results
percentDmax=deltaDmax*100
print('Gamma Index Pass Rate '+str(percentDmax) + '% - ' +str(DTA) + 'mm :'  +str(passRate) +' %')

      
    
# Plot profil or PDD with plotly      
trace1 = go.Scatter(
    x = refX2,
    y = refY2,
    mode = 'lines+markers', 
    name = 'measurement')

trace2 = go.Scatter(
    x = gateX2,
    y = gateY2,
    mode = 'lines+markers', 
    name = 'GATE')

#Plot gamma Index with plotly
trace3 = go.Scatter(
    x = refX2,
    y = gammaIndex,
    mode = 'markers', 
    name = 'gamma index GATE '+str(percentDmax) +'%/'+str(DTA) +'mm')


            
data = [trace1, trace2]
py.iplot(data, filename='scatter-mode')   


data1 = [trace3]
py.iplot(data1, filename='scatter-mode')   

nombre de points testés : 1185
nombre de points passant le test (<1) : 952.0
Gamma Index Pass Rate 3.0% - 3mm :80.3375527426 %


In [530]:
Xscale=-3
Yscale=1.0

In [541]:
#Create numpy ndarray from pandas dataFrames
#You can change headers
refX=df1['pas']
refY=df1['value']
emcX=df3['pas']
emcY=df3['value']

print type(df1['value'][1])
refX=refX-(refX.max()/2)
emcX=emcX-(emcX.max()/2)

# Compute Linear interpolation for simulated values and measured values according to your parameters (change 1000 values to change number of points)
if LinearInterpolation==True:
    #A=((refX.max()-refX.min())/NewStep)
    #B=((gateX.max()-gateX.min())/NewStep)
    refX2=np.linspace(refX.min(), refX.max(), 2000)
    refY2=np.interp(refX2, refX, refY)
    emcX2=np.linspace(emcX.min(), emcX.max(), 2000)
    emcY2=np.interp(emcX2, emcX, emcY)
else:
    refX2= refX
    refY2= refY
    emcX2= emcX
    emcY2= emcY
    

# Scaling for relative comparison
emcX2=emcX2+Xscale1
emcY2=emcY2*Yscale1



#create numpy ndarray for gamma comprison
deltaD=np.zeros(len(refY2))
gammaIndex2=np.zeros(len(emcX2))
gammaIndexTEMP=np.zeros(len(refX2))
drTEMP=np.zeros(len(refX2))
dr=np.zeros(len(refX2))

#compute Gamma Index 1D
for i in range(len(emcX2)):
    for j in range(len(refX2)):
        dr[j]=(emcX2[i]-refX2[j])
        deltaD[j]=emcY2[i]-refY2[j]
        gammaIndexTEMP[j]=m.sqrt(((deltaD[j]**2)/deltaDmax**2) + ((dr[j]**2)/DTA**2))
    #print gammaIndexTEMP.argmin()
    gammaIndex2[i]=gammaIndexTEMP.min()

#Compute Gamma Index Pass Rate (with threshold or not)
nok=0.0
a=0
if Threshold==True: 
    for i in range(len(gammaIndex2)):

        if emcY2[i]>0.10:
            a=a+1
            if gammaIndex2[i]<1:
                nok=nok+1
    print ('nombre de points testés : ' +str(a))
    print ('nombre de points passant le test (<1) : ' +str(nok))
    passRate=(nok/a)*100
else:
    nok=(gammaIndex2 < 1).sum()
    passRate=nok*100.0/len(gammaIndex2)

#print results
percentDmax=deltaDmax*100
print('Gamma Index Pass Rate '+str(percentDmax) + '% - ' +str(DTA) + 'mm :'  +str(passRate) +' %')

      
    
# Plot profil or PDD with plotly      

trace4 = go.Scatter(
    x = emcX2,
    y = emcY2,
    mode = 'lines+markers', 
    name = 'eMC')

#Plot gamma Index with plotly
trace5 = go.Scatter(
    x = refX2,
    y = gammaIndex2,
    mode = 'markers', 
    name = 'gamma index eMC '+str(percentDmax) +'%/'+str(DTA) +'mm')


            
data = [trace1, trace4]
py.iplot(data, filename='scatter-mode')   

layout = go.Layout(
    yaxis=dict(
        range = [0 , 5]), 
     )
data1 = [trace5]
fig0= go.Figure(data=data1, layout=layout)
py.iplot(fig0, filename='scatter-mode')   

<type 'numpy.float64'>
nombre de points testés : 801
nombre de points passant le test (<1) : 408.0
Gamma Index Pass Rate 3.0% - 3mm :50.936329588 %


#### Set X scaling and Y scaling  and launch again gamma index
##### Xscale=+5 or -5
##### Yscale=1.05 or 0.98

In [540]:
Xscale1=23
Yscale1=1


In [204]:
layout = go.Layout(
    xaxis=dict(
        range = [-90 , 90]), 
     )


data3 = [trace1, trace2, trace4]
fig1= go.Figure(data=data3, layout=layout)
py.iplot(fig1, filename='scatter-mode')   

layout = go.Layout(
    xaxis=dict(
        range = [-65 , 65]), 
     yaxis=dict(
        range = [0 , 7]))
data4 = [trace3, trace5]

fig2 = go.Figure(data=data4, layout=layout)
py.iplot(fig2, filename='scatter-mode')

diffGate=((refY2-gateY2)/refY2)*100
diffemc=((refY2-emcY2)/refY2)*100


trace6 = go.Scatter(
    x = refX2,
    y = diffGate,
    mode = 'markers', 
    name = 'diff gate'
)

trace7 = go.Scatter(
    x = refX2,
    y = diffemc,
    mode = 'markers', 
    name = 'diff emc'
)

layout = go.Layout(
    xaxis=dict(
        range = [-65 , 65]))
data5 = [trace6, trace7]

fig3 = go.Figure(data=data5, layout=layout)
py.iplot(fig3, filename='scatter-mode')

In [11]:
import scipy.optimize as optimize

#Create numpy ndarray from pandas dataFrames
#You can change headers
refX=df1['pas']
refY=df1['value']
gateX=df2['pas']
gateY=df2['value']




# Compute Linear interpolation for simulated values and measured values according to your parameters (change 1000 values to change number of points)
if LinearInterpolation==True:
    #A=((refX.max()-refX.min())/NewStep)
    #B=((gateX.max()-gateX.min())/NewStep)
    refX2=np.linspace(refX.min(), refX.max(), 2000)
    refY2=np.interp(refX2, refX, refY)
    gateX2=np.linspace(gateX.min(), gateX.max(), 2000)
    gateY2=np.interp(gateX2, gateX, gateY)
else:
    refX2= refX
    refY2= refY
    gateX2 = gateX
    gateY2= gateY2
    

# Scaling for relative comparison
#gateX2=gateX2+Xscale
#gateY2=gateY2*Yscale


def GammaIndex(scale):
    
    global gateX2
    global gateY2
    
    X, Y = scale
    
    print ('X : ' +str(X))
    print ('Y : ' +str(Y))
    # Scaling for relative comparison
    gateX2=gateX2+X
    gateY2=gateY2*Y
    
    #create numpy ndarray for gamma comprison
    deltaD=np.zeros(len(refY2))
    gammaIndex=np.zeros(len(gateX2))
    gammaIndexTEMP=np.zeros(len(refX2))
    drTEMP=np.zeros(len(refX2))
    dr=np.zeros(len(refX2))

    #compute Gamma Index 1D
    for i in range(len(gateX2)):
        for j in range(len(refX2)):
            dr[j]=(gateX2[i]-refX2[j])
            deltaD[j]=gateY2[i]-refY2[j]
            gammaIndexTEMP[j]=m.sqrt(((deltaD[j]**2)/deltaDmax**2) + ((dr[j]**2)/DTA**2))           
        gammaIndex[i]=gammaIndexTEMP.min()

    #Compute Gamma Index Pass Rate (with threshold or not)
    nok=0.0
    a=0
    if Threshold==True: 
        for i in range(len(gammaIndex)):

            if gateY2[i]>0.10:
                a=a+1
                if gammaIndex[i]<1:
                    nok=nok+1
        print ('nombre de points testés : ' +str(a))
        print ('nombre de points passant le test (<1) : ' +str(nok))
        passRate=(nok/a)*100
    else:
        nok=(gammaIndex < 1).sum()
        passRate=nok*100.0/len(gammaIndex)
        
    NopassRate=100.0-passRate
    #print results
    percentDmax=deltaDmax*100
    print('Gamma Index Pass Rate '+str(percentDmax) + '% - ' +str(DTA) + 'mm :'  +str(passRate) +' %')
    print('NO Pass Rate '+str(percentDmax) + '% - ' +str(DTA) + 'mm :'  +str(NopassRate) +' %')
    return NopassRate

initial= [1.0, 1.0]
res= optimize.minimize(GammaIndex, initial)

if res.success:
    fitted_params = result.x
    print(fitted_params)
else:
    raise ValueError(result.message)
    
# Plot profil or PDD with plotly      
trace1 = go.Scatter(
    x = refX2,
    y = refY2,
    mode = 'lines+markers', 
    name = 'measurement')

trace2 = go.Scatter(
    x = gateX2,
    y = gateY2,
    mode = 'lines+markers', 
    name = 'simulation')

#Plot gamma Index with plotly
trace3 = go.Scatter(
    x = refX2,
    y = gammaIndex,
    mode = 'markers', 
    name = 'gamma index'+str(percentDmax) +'%/'+str(DTA) +'mm')


            
data = [trace1, trace2]
py.iplot(data, filename='scatter-mode')   


data1 = [trace3]
py.iplot(data1, filename='scatter-mode')   

X : 1.0
Y : 1.0
nombre de points testés : 1277
nombre de points passant le test (<1) : 827.0
Gamma Index Pass Rate 3.0% - 3mm :64.7611589663 %
NO Pass Rate 3.0% - 3mm :35.2388410337 %
X : 1.0000000149
Y : 1.0
nombre de points testés : 1277
nombre de points passant le test (<1) : 782.0
Gamma Index Pass Rate 3.0% - 3mm :61.237274863 %
NO Pass Rate 3.0% - 3mm :38.762725137 %
X : 1.0
Y : 1.0000000149
nombre de points testés : 1277
nombre de points passant le test (<1) : 555.0
Gamma Index Pass Rate 3.0% - 3mm :43.4612372749 %
NO Pass Rate 3.0% - 3mm :56.5387627251 %
X : 1.0
Y : 1.0
nombre de points testés : 1277
nombre de points passant le test (<1) : 481.0
Gamma Index Pass Rate 3.0% - 3mm :37.6664056382 %
NO Pass Rate 3.0% - 3mm :62.3335943618 %
X : 0.835145283008
Y : 0.00354482173782
nombre de points testés : 0
nombre de points passant le test (<1) : 0.0


ZeroDivisionError: float division by zero

In [6]:
def f(params):
    # print(params)  # <-- you'll see that params is a NumPy array
    a, b = params # <-- for readability you may wish to assign names to the component variables
    global gateX2
    global gateY2
    gateX2=gateX2+a
    gateY2=gateY2*b
    h=max(gateX2+gateY2)
    return h

initial_guess = [1, 1]
result = optimize.minimize(f, initial_guess)
if result.success:
    fitted_params = result.x
    print(fitted_params)
else:
    raise ValueError(result.message)


ValueError: Desired error not necessarily achieved due to precision loss.